# 10G Ethernet core example
References:

https://github.com/Xilinx/embeddedsw/blob/master/XilinxProcessorIPLib/drivers/xxvethernet/src/xxxvethernet.c
https://docs.amd.com/r/en-US/pg210-25g-ethernet/Configuration-Register-Map-10G/25G-Ethernet-Subsystem

In [ ]:
import time
import sys
import xrfdc
import xrfclk
from rfsoc_qsfp_offload.overlay import Overlay

print("Initializing RFSoC 10G Ethernet Offload Overlay")
ol = Overlay(ignore_version=True)
#ol = Overlay(bitfile_name="/opt/bitstream/rfsoc_offload_10g_edge_5.bit",ignore_version=True)


# Wait for overlay to initialize
print("Waiting 5 sec for overlay to initialize")
time.sleep(5)
print("Initialized")

## Set ADC sample rate and center frequency

In [ ]:
## Set Reference clocks
lmx_freq=491.52

# Config file for lmk_freq = 245.76 defaults to RFSoC VCO clock
# xrfclk.set_ref_clks(lmk_freq=245.76, lmx_freq=lmx_freq)

# Config file for lmk_freq = 122.88 also set clock reference to external
xrfclk.set_ref_clks(lmk_freq=122.88, lmx_freq=lmx_freq)

In [ ]:
## Set ADC A and B capture configuration
ADC_TILE = 2       # ADC Tile 226
ADC_BLOCK = 0       # ADC Block 0
#ADC_SAMPLE_FREQUENCY = 1228.8  # MSps
ADC_SAMPLE_FREQUENCY = 1024  # MSps, default is 16x decimation
ADC_DECIMATION = 16 # Default, not actively set
ADC_PLL_FREQUENCY = lmx_freq   # MHz
ADC_FC = -1*93.3 # FM Band

pll_freq = ADC_PLL_FREQUENCY
fs = ADC_SAMPLE_FREQUENCY
tile = ADC_TILE
block=ADC_BLOCK
fc = ADC_FC

mixer_settings = {
            'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
            'EventSource':    xrfdc.EVNT_SRC_TILE,
            'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
            'Freq':           fc,
            'MixerMode':      xrfdc.MIXER_MODE_R2C,
            'MixerType':      xrfdc.MIXER_TYPE_FINE,
            'PhaseOffset':    0.0
        }

block = 0
ol.rfdc.adc_tiles[tile].DynamicPLLConfig(1, pll_freq, fs)
ol.rfdc.adc_tiles[tile].blocks[block].NyquistZone = 1
ol.rfdc.adc_tiles[tile].blocks[block].MixerSettings = mixer_settings
ol.rfdc.adc_tiles[tile].blocks[block].UpdateEvent(xrfdc.EVENT_MIXER)
ol.rfdc.adc_tiles[tile].SetupFIFO(True)

block = 1
ol.rfdc.adc_tiles[tile].DynamicPLLConfig(1, pll_freq, fs)
ol.rfdc.adc_tiles[tile].blocks[block].NyquistZone = 1
ol.rfdc.adc_tiles[tile].blocks[block].MixerSettings = mixer_settings
ol.rfdc.adc_tiles[tile].blocks[block].UpdateEvent(xrfdc.EVENT_MIXER)
ol.rfdc.adc_tiles[tile].SetupFIFO(True)

# Set UDP header to the sample frequency (denominator is already set to 16x)
ol.adc_to_udp_stream_A.register_map.SAMPLE_RATE_NUMERATOR_LSB = ADC_SAMPLE_FREQUENCY * 1e6
ol.adc_to_udp_stream_B.register_map.SAMPLE_RATE_NUMERATOR_LSB = ADC_SAMPLE_FREQUENCY * 1e6

## Example Configuration Code
### Configure XXV Ethernet Core

In [ ]:
# Stop UDP Stream
ol.adc_to_udp_stream_A.register_map.CTRL = 1
ol.adc_to_udp_stream_B.register_map.CTRL = 1
ol.adc_to_udp_stream_C.register_map.CTRL = 1
ol.adc_to_udp_stream_D.register_map.CTRL = 1

# Reset Ethernet core
eth_mmio = ol.xxv_ethernet_0.mmio
reg_map = ol.xxv_ethernet_0.register_map
tx_cfg_reg = reg_map.CONFIGURATION_TX_REG1
rx_cfg_reg = reg_map.CONFIGURATION_RX_REG1

# Stop Ethernet reciever 
tx_cfg_reg.ctl_tx_enable = 0
rx_cfg_reg.ctl_rx_enable = 0

reg_map.RESET_REG.tx_reset = 1
reg_map.RESET_REG.tx_serdes_reset = 1
reg_map.RESET_REG.rx_reset = 1
reg_map.RESET_REG.rx_serdes_reset = 1

print(f"TX Configuration: {tx_cfg_reg}")
print(f"RX Configuration: {rx_cfg_reg}")

# Set Ethernet core TX configration
tx_cfg_reg.ctl_tx_enable = 0
reg_map.RESET_REG.tx_reset = 1

# Enable FCS insertion by the TX core. If this bit is set to 0,
# the core does not add FCS to packet. If this bit is set to
# 1, the core calculates and adds the FCS to the packet.
# This input cannot be changed dynamically between
# packets.
tx_cfg_reg.ctl_tx_fcs_ins_enable = 1

# Enable FCS error checking at the AXI4-Stream interface
# by the TX core. This input only has effect when
# ctl_tx_fcs_ins_enable is Low. If this input is Low and a
# packet with bad FCS is being transmitted, it is not
# binned as good. If this input is High, a packet with bad
# FCS is binned as good.
tx_cfg_reg.ctl_tx_ignore_fcs = 0

# Transmit Idle code words. If this input is sampled as a 1,
# the TX path only transmits Idle code words. This input
# should be set to 1 when the partner device is sending
# RFI code words.
tx_cfg_reg.ctl_tx_send_idle = 0 

# Parity error response by the TX Core. If this bit is set to
# 0, the core does not take any action if any parity errors
# are detected. If this bit is set to 1, the core stomps the
# outgoing FCS (i.e., bit-wise inverse) and asserts
# stat_tx_bad_fcs.
tx_cfg_reg.ctl_tx_parity_err_response = 0

reg_map.RESET_REG.tx_reset = 0
reg_map.RESET_REG.tx_serdes_reset = 0
reg_map.RESET_REG.rx_reset = 0
reg_map.RESET_REG.rx_serdes_reset = 0

tx_cfg_reg.ctl_tx_enable = 1
print("Transmit enabled")

rx_cfg_reg.ctl_rx_enable = 1
print("Receive enabled")

# Ethernet core TX status register
print(f"TX_CFG_REG: {tx_cfg_reg}")

## Start on PPS Edge Example (Channel B)

In [ ]:
# Place all ADC channels in reset
ol.adc_to_udp_stream_A.register_map.CTRL = 1 # Set A control reg to 0x01
ol.adc_to_udp_stream_B.register_map.CTRL = 1 # Set B control reg to 0x01
ol.adc_to_udp_stream_C.register_map.CTRL = 1 # Set C control reg to 0x01
ol.adc_to_udp_stream_D.register_map.CTRL = 1 # Set D control reg to 0x01

# Wait for beginning of second to initiate capture
current_time = time.time()
start_time = current_time
while((current_time - int(current_time)) > .5):
    time.sleep(.1)
    current_time = time.time()

# Delay 100ms for PPS sync
time.sleep(.1)
current_time = time.time()

# Set enable on next pps capture 
ol.adc_to_udp_stream_B.register_map.CTRL = 3 # Set B control reg to 0x11

# Set start time in UDP Header
current_time_s = int(current_time) + 1 # Stream starts PPS edge
samples_since_epoch = int(current_time_s * ((ADC_SAMPLE_FREQUENCY * 1e6)/ ADC_DECIMATION))
samples_since_epoch_lsb = samples_since_epoch & 0xFFFFFFFF
samples_since_epoch_msb = samples_since_epoch >> 32
ol.adc_to_udp_stream_B.register_map.SAMPLE_IDX_OFFSET_LSB = samples_since_epoch_lsb
ol.adc_to_udp_stream_B.register_map.SAMPLE_IDX_OFFSET_MSB = samples_since_epoch_msb

while(int(ol.adc_to_udp_stream_B.register_map.PPS_COUNTER) < 1):
    time.sleep(.01)
end_time = time.time()

print(f"Start time: {start_time} Enable time: {current_time}")
print(f"Set time: {current_time_s} End time: {end_time}")


In [ ]:
pps_count_a = ol.adc_to_udp_stream_A.register_map.PPS_COUNTER
pps_count_b = ol.adc_to_udp_stream_B.register_map.PPS_COUNTER
print(f"PPS Count A: {int(pps_count_a)} B: {int(pps_count_b)}")

### Check 10G Ethernet core statistics

In [ ]:
# Capture statistics
import time
reg_map.TICK_REG = 1
time.sleep(5)
reg_map.TICK_REG =1 
print("Packets          : %s %s" % (str(reg_map.STAT_TX_TOTAL_PACKETS_MSB),str(reg_map.STAT_TX_TOTAL_PACKETS_LSB)))
print("Good Packets     : %s %s" % (str(reg_map.STAT_TX_TOTAL_GOOD_PACKETS_MSB),str(reg_map.STAT_TX_TOTAL_GOOD_PACKETS_LSB)))
print("Small Packets    : %s %s" % (str(reg_map.STAT_TX_PACKET_SMALL_MSB),str(reg_map.STAT_TX_PACKET_SMALL_LSB)))
print("Large Packets    : %s %s" % (str(reg_map.STAT_TX_PACKET_LARGE_MSB),str(reg_map.STAT_TX_PACKET_LARGE_LSB)))
print("64               : %s %s" % (str(reg_map.STAT_TX_PACKET_64_BYTES_MSB),str(reg_map.STAT_TX_PACKET_64_BYTES_LSB)))
print("65-127           : %s %s" % (str(reg_map.STAT_TX_PACKET_65_127_BYTES_MSB),str(reg_map.STAT_TX_PACKET_65_127_BYTES_LSB)))
print("256-511          : %s %s" % (str(reg_map.STAT_TX_PACKET_256_511_BYTES_MSB),str(reg_map.STAT_TX_PACKET_256_511_BYTES_LSB)))
print("512-1023         : %s %s" % (str(reg_map.STAT_TX_PACKET_512_1023_BYTES_MSB),str(reg_map.STAT_TX_PACKET_512_1023_BYTES_LSB)))
print("1024-1518        : %s %s" % (str(reg_map.STAT_TX_PACKET_1024_1518_BYTES_MSB),str(reg_map.STAT_TX_PACKET_1024_1518_BYTES_LSB)))
print("Total bytes      : %s %s" % (str(reg_map.STAT_TX_TOTAL_BYTES_MSB),str(reg_map.STAT_TX_TOTAL_BYTES_LSB)))
print("Good bytes       : %s %s" % (str(reg_map.STAT_TX_TOTAL_GOOD_BYTES_MSB),str(reg_map.STAT_TX_TOTAL_GOOD_BYTES_LSB)))
print("FIFO Error       : %s %s" % (str(reg_map.STAT_TX_FRAME_ERROR_MSB),str(reg_map.STAT_TX_FRAME_ERROR_LSB)))
print("Frame Error      : %s %s" % (str(reg_map.STAT_TX_FRAME_ERROR_MSB),str(reg_map.STAT_TX_FRAME_ERROR_LSB)))
print("FCS   Error      : %s %s" % (str(reg_map.STAT_TX_BAD_FCS_MSB),str(reg_map.STAT_TX_BAD_FCS_LSB)))
print("Unicast Packets  : %s %s" % (str(reg_map.STAT_TX_UNICAST_MSB),str(reg_map.STAT_TX_UNICAST_LSB)))
print("Multicast Packets: %s %s" % (str(reg_map.STAT_TX_MULTICAST_MSB),str(reg_map.STAT_TX_MULTICAST_LSB)))
print("Broadcast Packets: %s %s" % (str(reg_map.STAT_TX_BROADCAST_MSB),str(reg_map.STAT_TX_BROADCAST_LSB)))

print("Bad FCS          : %s %s" % (str(reg_map.STAT_TX_BAD_FCS_MSB),str(reg_map.STAT_TX_BAD_FCS_LSB)))
print("Bad Frame        : %s %s" % (str(reg_map.STAT_TX_FRAME_ERROR_MSB),str(reg_map.STAT_TX_FRAME_ERROR_LSB)))

### UDP Streaming core stats

In [ ]:
ol.adc_to_udp_stream_B.register_map.RECEIVED_COUNTER

### Configure UDP Streaming core
Stop here if using default values, configuration changes may break stream to receiving devices

Enable USER_RESET (CTRL = 0x01) before changing any configuration values

In [ ]:
print("Default Dest MAC MSB: %08X" % ol.adc_to_udp_stream_A.register_map.ETH_DST_MAC_MSB)
print("Default Dest MAC LSB: %08X" % ol.adc_to_udp_stream_A.register_map.ETH_DST_MAC_LSB)

# Set Destination MAC to MEP-B09E: 6c:92:bf:42:52:12
# Header is updated on MSB write, writes must be in order
ol.udp_stream_A.register_map.ETH_DST_MAC_LSB = 0x6c92bf42
ol.udp_stream_A.register_map.ETH_DST_MAC_MSB = 0x00005212

ol.adc_to_udp_stream_A.register_map.ETH_DST_MAC_LSB = 0xbf425212
ol.adc_to_udp_stream_A.register_map.ETH_DST_MAC_MSB = 0x00006c92

print("New Dest MAC MSB    : %08X" % ol.adc_to_udp_stream_0.register_map.ETH_DST_MAC_MSB)
print("New Dest MAC LSB    : %08X" % ol.adc_to_udp_stream_0.register_map.ETH_DST_MAC_LSB)

In [ ]:
print("Default Destination IP: %08X" % ol.adc_to_udp_stream_A.register_map.IP_DST_ADDR)
#192.168.4.1 (default)
ol.adc_to_udp_stream_0.register_map.IP_DST_ADDR = 0xc0A80401
print("New Destination IP    : %08X" % ol.adc_to_udp_stream_A.register_map.IP_DST_ADDR)

In [ ]:
print("Default Source IP: %08X" % ol.adc_to_udp_stream_B.register_map.IP_SRC_ADDR)
#192.168.4.8
ol.adc_to_udp_stream_B.register_map.IP_SRC_ADDR = 0xc0A80408
print("New Source IP    : %08X" % ol.adc_to_udp_stream_B.register_map.IP_SRC_ADDR)


In [ ]:
print("Default Source PORT: %08X" % ol.adc_to_udp_stream_A.register_map.IP_SRC_PORT)
#1020
ol.adc_to_udp_stream_A.register_map.IP_SRC_PORT = 10
print("New Source PORT    : %08X" % ol.adc_to_udp_stream_A.register_map.IP_SRC_PORT)


In [ ]:
# 5-bit address bus?
print("Default Destination PORT: %08X" % ol.udp_stream_A.register_map.IP_DST_PORT)
#1020
ol.udp_stream_0.register_map.IP_DST_PORT = 1021
print("New Destination PORT    : %08X" % ol.udp_stream_A.register_map.IP_DST_PORT)


In [ ]:
ol.adc_to_udp_stream_A.register_map.CTRL = 1
ol.adc_to_udp_stream_B.register_map.CTRL = 1

In [ ]:
print(f"A Control: {ol.adc_to_udp_stream_A.register_map.CTRL}")
print(f"B Control: {ol.adc_to_udp_stream_B.register_map.CTRL}")